In [3]:
import os
from glob import glob

from pydub import AudioSegment
from pydub.playback import play

In [4]:
source_dir = os.path.join('./animal_meeting_1509_librivox')

In [5]:
sources = sorted(glob(os.path.join(source_dir, '*')))
print(sources)

['./animal_meeting_1509_librivox/animalmeeting_01_ahn_64kb.mp3', './animal_meeting_1509_librivox/animalmeeting_02_ahn_64kb.mp3', './animal_meeting_1509_librivox/animalmeeting_03_ahn_64kb.mp3', './animal_meeting_1509_librivox/animalmeeting_04_ahn_64kb.mp3', './animal_meeting_1509_librivox/animalmeeting_05_ahn_64kb.mp3', './animal_meeting_1509_librivox/animalmeeting_06_ahn_64kb.mp3']


# speed up

In [6]:
audio = AudioSegment.from_file(sources[0])

In [7]:
demo_audio = audio[:30000]

# Now we can apply the same process as before:
octaves = 0.5  # This time we'll only increase the pitch by a half octave
new_sample_rate = int(demo_audio.frame_rate * (2.0 ** octaves))
print(new_sample_rate)

31183


In [8]:
# Make sure the new sample rate is one of the standard rates
# Common standard sample rates include 8000, 11025, 16000, 22050, 32000, 44100, and 48000 Hz.
standard_rates = [8000, 11025, 16000, 22050, 32000, 44100, 48000]
closest_rate = min(standard_rates, key=lambda x: abs(x - new_sample_rate))
if new_sample_rate not in standard_rates:
    new_sample_rate = closest_rate

In [9]:
high_pitch_audio = demo_audio._spawn(demo_audio.raw_data, overrides={'frame_rate': new_sample_rate})
fast_audio = high_pitch_audio.speedup(playback_speed=2)

In [12]:
play(demo_audio)

In [13]:
play(fast_audio)

# librosa

In [1]:
import librosa
import soundfile as sf

In [2]:
# Load the audio file
y, sr = librosa.load(sources[0])

NameError: name 'sources' is not defined

In [ ]:
n_steps = 6  # Positive values raise the pitch, negative values lower it.
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

In [ ]:
shifted_file = "pitch_shifted_audio.wav"
sf.write(shifted_file, y_shifted, sr)

In [32]:
audio = AudioSegment.from_file(shifted_file)

In [33]:
demo_audio = audio[:30000]
play(demo_audio)

# ...

In [14]:
# Shift the pitch up by 6 semitones (half an octave)
octaves = 6 / 12  # semitones to octaves
new_sample_rate = int(audio.frame_rate * (2.0 ** octaves))

In [15]:
# Change the audio sample rate, this also changes the pitch
chipmunk = audio._spawn(audio.raw_data, overrides={'frame_rate': new_sample_rate})

In [16]:
# Calculate the new duration
new_duration = chipmunk.duration_seconds / (2.0 ** octaves)

In [17]:
import subprocess

In [18]:
temp_file = "temp_pitch_shifted.wav"
chipmunk.export(temp_file, format="wav")

# Use the external soundstretch command to adjust the speed to the original
stretched_file = "stretched_audio.wav"
cmd = f"soundstretch {temp_file} {stretched_file} -tempo={(2.0 ** octaves - 1) * 100}"

# Run the command
subprocess.run(cmd, shell=True)

# Clean up the temporary file
os.remove(temp_file)


   SoundStretch v2.3.2 -  Copyright (c) Olli Parviainen
author e-mail: <oparviai@iki.fi> - WWW: http://www.surina.net/soundtouch

This program is subject to (L)GPL license. Run "soundstretch -license" for
more information.

Uses 32bit floating point sample type in processing.

Processing the file with the following changes:
  tempo change = +41.4214 %
  pitch change = +0 semitones
  rate change  = +0 %

Working...Done!


In [19]:
# Now load the stretched audio
stretched_chipmunk = AudioSegment.from_file(stretched_file)

In [20]:
print(stretched_chipmunk.frame_rate)

31183


In [21]:
# Convert to a standard sample rate if necessary
standard_sample_rate = 44100
if stretched_chipmunk.frame_rate != standard_sample_rate:
    stretched_chipmunk = stretched_chipmunk.set_frame_rate(standard_sample_rate)

In [44]:
play(demo_audio)

In [22]:
# Now try playing it again
play(stretched_chipmunk)

# soundstretch pitch and tempo

In [24]:
# Replace 'input.mp3' with your mp3 file path
input_mp3 = sources[0]
original_wav = 'input.wav'  # Output file name

# Load MP3 file
audio_segment = AudioSegment.from_mp3(input_mp3)

# Export as WAV
audio_segment.export(original_wav, format="wav")

<_io.BufferedRandom name='input.wav'>

In [25]:
temp_file = "output_pitch_shifted.wav"
# Run the pitch shift command
result = subprocess.run(
    ["soundstretch", original_wav, temp_file, "-pitch=12"],
    capture_output=True,
    text=True
)
print(result.stdout)
print(result.stderr)



   SoundStretch v2.3.2 -  Copyright (c) Olli Parviainen
author e-mail: <oparviai@iki.fi> - WWW: http://www.surina.net/soundtouch

This program is subject to (L)GPL license. Run "soundstretch -license" for
more information.

Uses 32bit floating point sample type in processing.

Processing the file with the following changes:
  tempo change = +0 %
  pitch change = +12 semitones
  rate change  = +0 %

Working...Done!



In [26]:
output_file = "output_final.wav"
# Run the tempo adjustment command
result = subprocess.run(
    ["soundstretch", temp_file, output_file, "-tempo=-50"],
    capture_output=True,
    text=True
)
print(result.stdout)
print(result.stderr)

# Load the final audio for playback or further processing
stretched_chipmunk = AudioSegment.from_file(output_file)



   SoundStretch v2.3.2 -  Copyright (c) Olli Parviainen
author e-mail: <oparviai@iki.fi> - WWW: http://www.surina.net/soundtouch

This program is subject to (L)GPL license. Run "soundstretch -license" for
more information.

Uses 32bit floating point sample type in processing.

Processing the file with the following changes:
  tempo change = -50 %
  pitch change = +0 semitones
  rate change  = +0 %

Working...Done!



In [27]:
# Attempt to play the adjusted audio
play(stretched_chipmunk)

In [29]:
pitch_chipmunk = AudioSegment.from_file(temp_file)
play(pitch_chipmunk)

In [28]:
play(demo_audio)